In [1]:
# Automatic reload imports
%load_ext autoreload
%autoreload 2

In [2]:
# Run notebooks as it would run if was one folder upper in the tree, enabling direct reference to source files
import sys
sys.path.insert(0, '../')

In [3]:
import torch
import torch.nn as nn
import numpy as np
from pprint import pprint

from src.utils.level_dict_sem_eval import hierarchy
from src.model.hierarchical_loss_semeval import HierarchicalLossNetwork

In [4]:
if torch.cuda.is_available():
        available_device = "cuda" 
else:
    available_device = 'cpu'
device = torch.device(available_device)

print(f'{torch.cuda.device_count()} GPUs available.')
print(f'Torch is using {device}!')

2 GPUs available.
Torch is using cuda!


In [5]:
hloss = HierarchicalLossNetwork(
    hierarchical_labels=hierarchy,
    device=device,
    p_loss=3,
    alpha=1,
    beta=1
)

In [6]:
pprint(hierarchy)

{'Persuasion': {'Ethos (level 1)': {'Ad Hominem (level 2)': {'Ad Hominem (level 3)': {'Doubt': {},
                                                                                      'Name Calling': {},
                                                                                      'Reductio ad Hitlerum': {},
                                                                                      'Smears': {},
                                                                                      'Whataboutism - Ad Hominem (level 3)': {}}},
                                    'Ethos (level 2)': {'Ethos (level 3)': {'Appeal to Authority - Ethos (level 3)': {},
                                                                            'Bandwagon - Ethos (level 3)': {},
                                                                            'Glittering Generalities': {},
                                                                            'Transfer - Ethos (level 3)': {}}}}

In [7]:
pprint(hloss.hierarchical_index)

{0: {0: {0: {0: {0: {}, 1: {}, 2: {}, 3: {}, 4: {}}},
         1: {1: {5: {}, 6: {}, 7: {}, 8: {}}}},
     1: {2: {2: {9: {}, 10: {}, 11: {}, 12: {}, 13: {}, 14: {}}}},
     2: {3: {3: {15: {}, 16: {}, 17: {}, 18: {}, 19: {}}},
         4: {4: {20: {}, 21: {}}},
         5: {5: {22: {}, 23: {}, 24: {}}, 6: {25: {}, 26: {}, 27: {}}}}}}


In [12]:
hloss.check_hierarchy(['Pathos (level 1)'], ['Pathos (level 2)'],hierarchy_representation='label')

[True]

In [20]:
class_labels_by_level = hloss.get_class_labels_by_level()
hloss.label_to_index(class_labels_by_level)

[[0],
 [0, 1, 2],
 [0, 1, 2, 3, 4, 5],
 [0, 1, 2, 3, 4, 5, 6],
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27]]

In [22]:
hloss

TypeError: HierarchicalLossNetwork.get_children_by_index() missing 1 required positional argument: 'tree'

In [8]:
def make_fake_predictions(predictions_index, numeric_hierarchy, device):
    n_classes = {}
    n_classes['superclass'] = len(list(numeric_hierarchy.keys()))
    n_classes['subclass'] = len(np.ravel(list(hloss.numeric_hierarchy.values())))

    fake_predictions = {}
    fake_predictions['superclass'] = []
    fake_predictions['subclass'] = []

    for class_type in ['superclass', 'subclass']:
        for example in predictions_index:
            class_index = example[0 if class_type=='superclass' else 1]
            fake_prediction = np.zeros(n_classes[class_type])
            fake_prediction[class_index] = 1

            fake_predictions[class_type].append(fake_prediction)
        
        
        fake_predictions[class_type] = torch.Tensor(fake_predictions[class_type])
        fake_predictions[class_type] = fake_predictions[class_type].to(device)

    return [fake_predictions['superclass'], fake_predictions['subclass']]

def make_fake_labels(labels, device):
    tensor_labels = torch.Tensor(labels).to(device)
    tensor_labels = [tensor_labels[:,0], tensor_labels[:,1]]
    return tensor_labels

In [13]:
predictions_index = [
    [0, 55],  # Acerto; acertou superclasse, acertou subclasse, acertou dependencia
    [0, 30],  # Erro;   acertou superclasse, errou   subclasse, acertou dependencia
    [0, 32],  # Erro;   acertou superclasse, errou   subclasse, errou   dependencia
    [1, 55],  # Erro;   errou   superclasse, acertou subclasse, errou   dependencia
    [1, 32],  # Erro;   errou   superclasse, errou   subclasse, acertou dependencia
    [1, 62],  # Erro;   errou   superclasse, errou   subclasse, errou   dependencia
              # Erro;   acertou superclasse, acertou subclasse, errou   dependencia <- impossivel pois se acertar os dois acerta a dependencia
              # Erro;   errou   superclasse, acertou subclasse, acertou dependencia <- impossivel
]
labels = [
    [0, 55],
    [0, 55],
    [0, 55],
    [0, 55],
    [0, 55],
    [0, 55],
]

error_type = [
    'Acerto; acertou superclasse, acertou subclasse, acertou dependencia',
    'Erro;   acertou superclasse, errou   subclasse, acertou dependencia',
    'Erro;   acertou superclasse, errou   subclasse, errou   dependencia',
    'Erro;   errou   superclasse, acertou subclasse, errou   dependencia',
    'Erro;   errou   superclasse, errou   subclasse, acertou dependencia',
    'Erro;   errou   superclasse, errou   subclasse, errou   dependencia'
]

In [14]:
fake_predictions = make_fake_predictions(predictions_index, hloss.numeric_hierarchy, device)
fake_predictions

[tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.]], device='cuda:0'),
 tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [15]:
superclasses_predictions = torch.argmax(nn.Softmax(dim=1)(fake_predictions[0]), dim=1)
sublcasses_predictions = torch.argmax(nn.Softmax(dim=1)(fake_predictions[1]), dim=1)

print(f'superclasses_predictions: {superclasses_predictions}')
print(f'sublcasses_predictions: {sublcasses_predictions}')

fake_labels = make_fake_labels(labels, device)
print(f'fake_labels: {fake_labels}')

superclasses_predictions: tensor([0, 0, 0, 1, 1, 1], device='cuda:0')
sublcasses_predictions: tensor([55, 30, 32, 55, 32, 62], device='cuda:0')
fake_labels: [tensor([0., 0., 0., 0., 0., 0.], device='cuda:0'), tensor([55., 55., 55., 55., 55., 55.], device='cuda:0')]


In [16]:
hloss.calculate_lloss(fake_predictions, fake_labels)

RuntimeError: "nll_loss_forward_reduce_cuda_kernel_2d_index" not implemented for 'Float'

In [17]:
# l_loss = hloss.calculate_lloss(fake_predictions, fake_labels)
# print(f'LLoss: {lloss}')

d_loss = hloss.calculate_dloss(fake_predictions, fake_labels)
d_loss


tensor(12., device='cuda:0')

In [23]:
for i in range(len(predictions_index)):
    fake_predictions = make_fake_predictions([predictions_index[i]], hloss.numeric_hierarchy, device)
    fake_labels = make_fake_labels([labels[i]], device)

    d_loss = hloss.calculate_dloss(fake_predictions, fake_labels)
    
    print(f'# Teste {i+1} - label: {labels[i]} | prediction: {predictions_index[i]}')
    print(error_type[i])
    print(f'Dloss: {d_loss}\n')


# Teste 1 - label: [0, 55] | prediction: [0, 55]
Acerto; acertou superclasse, acertou subclasse, acertou dependencia
Dloss: 0.0

# Teste 2 - label: [0, 55] | prediction: [0, 30]
Erro;   acertou superclasse, errou   subclasse, acertou dependencia
Dloss: 0.0

# Teste 3 - label: [0, 55] | prediction: [0, 32]
Erro;   acertou superclasse, errou   subclasse, errou   dependencia
Dloss: 2.0

# Teste 4 - label: [0, 55] | prediction: [1, 55]
Erro;   errou   superclasse, acertou subclasse, errou   dependencia
Dloss: 2.0

# Teste 5 - label: [0, 55] | prediction: [1, 32]
Erro;   errou   superclasse, errou   subclasse, acertou dependencia
Dloss: 0.0

# Teste 6 - label: [0, 55] | prediction: [1, 62]
Erro;   errou   superclasse, errou   subclasse, errou   dependencia
Dloss: 8.0



In [ ]:
predictions_index[0]

[0, 55]